Tarefa
Utilize o dataset "Titanic - Machine Learning from Disaster" para realizar essa tarefa: https://www.kaggle.com/competitions/titanic/data

Um empresário do ramo de cruzeiros está procurando expandir seus negócios, mas precisa entender melhor seu público-alvo. Ele tem em mãos os dados do "Titanic - Machine Learning from Disaster", que contém informações sobre passageiros do navio Titanic. Com base nesses dados, o empresário pode tomar decisões mais informadas sobre como atrair e atender melhor seus clientes:

In [1]:
#Cria sessão Spark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import pyspark.sql.functions as F

In [2]:
#Instância Spark
spark = SparkSession.builder.appName('Curso Pyspark') \
         .config('spark.sql.repl.eagerEval.enabled', True) \
         .getOrCreate()      

In [3]:
# lendo um arquivo parquet
df = spark.read.csv('./DATASETS/Desafio/titanic.csv',sep=',', header=True)

In [4]:
df.show(20, truncate=0)

+-----------+--------+------+-------------------------------------------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|Name                                                   |Sex   |Age |SibSp|Parch|Ticket          |Fare   |Cabin|Embarked|
+-----------+--------+------+-------------------------------------------------------+------+----+-----+-----+----------------+-------+-----+--------+
|1          |0       |3     |Braund, Mr. Owen Harris                                |male  |22  |1    |0    |A/5 21171       |7.25   |NULL |S       |
|2          |1       |1     |Cumings, Mrs. John Bradley (Florence Briggs Thayer)    |female|38  |1    |0    |PC 17599        |71.2833|C85  |C       |
|3          |1       |3     |Heikkinen, Miss. Laina                                 |female|26  |0    |0    |STON/O2. 3101282|7.925  |NULL |S       |
|4          |1       |1     |Futrelle, Mrs. Jacques Heath (Lily May Peel)           |female|35  |1  

1 - O empresário quer saber se há alguma correlação entre a idade dos passageiros do Titanic e a classe em que estavam viajando. Ele gostaria de saber se os passageiros mais velhos tendiam a viajar em classes superiores ou inferiores. Como você poderia ajudá-lo a responder essa pergunta utilizando o conjunto de dados do Titanic?

In [5]:
(
df
.groupBy('Pclass')
.agg(F.round(F.avg('Age'),0).alias('Average_Age'))
.orderBy('Pclass')
.show(20, truncate=0)
)

+------+-----------+
|Pclass|Average_Age|
+------+-----------+
|1     |38.0       |
|2     |30.0       |
|3     |25.0       |
+------+-----------+




2 - O empresário está interessado em criar um programa de fidelidade para seus clientes, mas não tem certeza de qual abordagem seria mais eficaz. Ele gostaria de saber se há alguma relação entre o gênero dos passageiros do Titanic e a sobrevivência no navio.

In [6]:
(
df
.groupBy('Sex')
.agg(F.round(F.avg('Survived'),0).alias('Average_Survived'))
.orderBy('Sex')
.show(20, truncate=0)
)

+------+----------------+
|Sex   |Average_Survived|
+------+----------------+
|female|1.0             |
|male  |0.0             |
+------+----------------+



3 - Qual a probabilidade de sobrevivência de um passageiro de primeira classe com menos de 20 anos?

In [12]:
survival_rate = (
df
.filter((F.col('Pclass') == 1) & (F.col('Age') < 20))
.agg(F.avg('Survived').alias('Survival_Rate'))
.collect()[0]['Survival_Rate']
)
print(f'A probabilidade de sobrevivência de um passageiro de primeira classe com menos de 20 anos é de {survival_rate * 100:.2f}%')

A probabilidade de sobrevivência de um passageiro de primeira classe com menos de 20 anos é de 80.95%


4 - Quais as principais características em comum entre os sobreviventes do desastre do Titanic?

In [16]:
print("Sobreviventes por Classe:")
(
df
.filter(col('Survived') == 1)
.groupBy('Pclass').count().orderBy('Pclass')
.show(20, truncate=0)
)

Sobreviventes por Classe:
+------+-----+
|Pclass|count|
+------+-----+
|1     |136  |
|2     |87   |
|3     |119  |
+------+-----+



In [17]:
print("Sobreviventes por gênero:")
(
df
.filter(col('Survived') == 1)
.groupBy('Sex').count().orderBy('Sex')
.show(20, truncate=0)
)

Sobreviventes por gênero:
+------+-----+
|Sex   |count|
+------+-----+
|female|233  |
|male  |109  |
+------+-----+



In [22]:
print("Sobreviventes por idade (média de idade):")
(
df
.filter(col('Survived') == 1)
.agg(F.round(F.avg('Age'),0).alias('Survived_Average_Age'))
.show(20, truncate=0)
)

Sobreviventes por idade (média de idade):
+--------------------+
|Survived_Average_Age|
+--------------------+
|28.0                |
+--------------------+




5 - É possível identificar alguma correlação entre a idade e a taxa de sobrevivência dos passageiros que embarcaram em Southampton?

In [35]:
(
df
.filter(col('Embarked') == 'S')
.groupBy('Age')
.agg(F.round(F.avg('Survived'),2).alias('Survival_Rate_Southampton'))
.orderBy('Age')
.show(20, truncate=0)
)

AttributeError: 'GroupedData' object has no attribute 'orderBy'